In [267]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import re

# handing_missing_data

In [14]:
s1 = pd.Series(['shenzhen','guangzhou',np.nan,'shanxi'])
s1

0     shenzhen
1    guangzhou
2          NaN
3       shanxi
dtype: object

In [3]:
s1.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [12]:
s1[0] = None
s1.fillna('a',inplace = True)  # 默认状态是非原址的

In [15]:
s1[s1.notnull()]

0     shenzhen
1    guangzhou
3       shanxi
dtype: object

In [17]:
df1 = pd.DataFrame([[1.,6.5,3],[1,np.nan,np.nan],
                    [np.nan,np.nan,np.nan],[np.nan,6.5,3]])
df1

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [24]:
df1.dropna(how = 'any',axis = 0,thresh = 4) # thresh最少含有几个不是NA

,0,1,2


In [25]:
df1.fillna(0)

,0,1,2
0,1.0,6.5,3.0
1,1.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,6.5,3.0


In [26]:
df1.fillna({1:0.5,2:0.3})

,0,1,2
0,1.0,6.5,3.0
1,1.0,0.5,0.3
2,NaN,0.5,0.3
3,NaN,6.5,3.0


In [29]:
df1.mean()

0    1.0
1    6.5
2    3.0
dtype: float64

In [28]:
df1.fillna(df1.mean())

,0,1,2
0,1.0,6.5,3.0
1,1.0,6.5,3.0
2,1.0,6.5,3.0
3,1.0,6.5,3.0


# data transformation

In [36]:
df2 = pd.DataFrame({'k1':['one','two'] * 3 + ['two'],
                   'k2':[1,1,2,3,3,4,4]})
df2

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [31]:
df2.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [40]:
df2.drop(df2[df2.duplicated()].index) # 还有下面这个简单的方法

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [46]:
df2.drop_duplicates(['k2','k1'],keep = 'last') # 在列k1,k2上挑出重复项，
#重复项中keep保留哪个？

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
6,two,4


In [60]:
df3 = pd.DataFrame({'food':['bacon','pulled pork','bacon','Pastrami','corned beef',
                           'Bacon','pastrami','honey ham','nova lox'],
                   'ounces':[4,3,12,6,7.5,8,3,5,6]})
df3

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [56]:
meat_to_animal = {
    'bacon':'pig',
    'pulled pork':'pig',
    'pastrami':'cow',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salmon'
}

In [61]:
lowercase = df3.food.str.lower()

In [62]:
df3['animal'] = lowercase.map(meat_to_animal)

In [67]:
df3.food.map(lambda x :meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [68]:
s2 = pd.Series([1.,-999,2.,-999,-1000,3.])

In [69]:
s2.replace(-999,np.nan) # s2.replace({-999:np.nan})

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [71]:
s2.replace([-999,-1000],[np.nan,0]) # 或者传入一个dict

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [72]:
s2.replace({-999:np.nan,-1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [74]:
df4 = pd.DataFrame(np.arange(12).reshape((3,4)),
                  index = ['ohio','colorada','new york'],
                  columns = ['one','two','three','four'])
transform = lambda x :x[:4].upper()
df4.index = df4.index.map(transform)

In [75]:
df4

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [77]:
df4.rename(index = str.title,columns = str.upper) # 不改变原来的df4，除非传入inplace

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [79]:
df4.rename(index = {'OHIO':'INDIANA'},columns = {'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [82]:
ages = [1,20,4,45,67,8,14,23,29,37]
bins = [1,18,36,54,72,90]
cats = pd.cut(ages,bins) # codes 标签化

In [108]:
len(ages)

10

In [84]:
cats.codes

array([-1,  1,  0,  2,  3,  0,  0,  1,  1,  2], dtype=int8)

In [90]:
df4 = pd.DataFrame({'ages':ages,'labels':cats.codes + 1})

In [94]:
df4['labels'].value_counts()

2    3
1    3
3    2
4    1
0    1
Name: labels, dtype: int64

In [124]:
group_name = ['baby','tenager','younth','年轻','middle']
cats = pd.cut(ages,5,precision = 2,labels = group_name)

In [126]:
cats

[baby, tenager, baby, 年轻, middle, baby, baby, tenager, younth, younth]
Categories (5, object): [baby < tenager < younth < 年轻 < middle]

In [129]:
pd.DataFrame({'ages':ages,'labels':cats.codes + 1,'labels1':cats})

,ages,labels,labels1
0,1,1,baby
1,20,2,tenager
2,4,1,baby
3,45,4,年轻
4,67,5,middle
5,8,1,baby
6,14,1,baby
7,23,2,tenager
8,29,3,younth
9,37,3,younth


In [134]:
df5 = DataFrame(np.random.randn(1000,4))

In [137]:
df5[(np.abs(df5) > 3).any(1)] # 挑出行中有数大于3的所有行

,0,1,2,3
116,0.404248,-0.332906,-4.006717,-0.377880
125,-0.100531,-3.158639,-0.575563,0.273186
200,0.385274,0.263029,-3.997954,-0.685039
246,-1.571625,0.697506,3.111210,1.550039
448,-3.037058,-0.160333,-0.281804,-0.446305
612,-3.234758,-1.766285,0.572202,-0.854130
656,-1.850686,0.330825,-3.177800,-1.286274
657,0.769812,0.325894,-1.634379,3.192963
663,-1.648834,0.297200,-0.356612,3.603028
750,-1.732892,-1.626523,0.386384,-3.283828


In [141]:
df5[np.abs(df5) > 1] = np.sign(df5) * 3 # sign 签名：-1，0，1

In [156]:
df6 = DataFrame(np.arange(12).reshape((4,3)))
sampler = np.random.permutation(3) # permutation 生成随机数列

In [157]:
sampler

array([1, 0, 2])

In [158]:
df6.take(sampler,axis = 'columns')

,1,0,2
0,1,0,2
1,4,3,5
2,7,6,8
3,10,9,11


In [159]:
np.random.permutation(df6) # 只在第一个索引上随机

array([[ 9, 10, 11],
       [ 3,  4,  5],
       [ 0,  1,  2],
       [ 6,  7,  8]])

In [165]:
df6.sample(n = 7,replace = True).drop_duplicates()

,0,1,2
3,9,10,11
0,0,1,2
2,6,7,8
1,3,4,5


In [172]:
pd.get_dummies(df6.iloc[0]) # 对离散特征进行one_hot编码

,0,1,2
0,1,0,0
1,0,1,0
2,0,0,1


In [173]:
df7 = DataFrame({'key':list('bbacab'),
                'data1':range(6)})

In [179]:
dummies = pd.get_dummies(df7.key,prefix = 'key')

In [187]:
df7[['data1']].join(dummies) # 在行索引上合并

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [195]:
mnames = ['movie_id','title','genres']
movies = pd.read_table('datasets\\movielens\\movies.dat',sep = '::',
                       header = None,names = mnames,engine = 'python')

In [198]:
movies.head(10)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [206]:
genr_list = []
for i in movies.genres:
    genr_list.extend(i.split('|'))

genr_list = pd.unique(genr_list)

In [209]:
dummies2 = DataFrame(np.zeros((len(movies),len(genr_list))),columns = genr_list)

In [219]:
dummies2.columns.get_indexer(movies.iloc[0,2].split('|'))

array([0, 1, 2], dtype=int64)

In [212]:
for i,gen in enumerate(movies.genres):
    indices = dummies2.columns.get_indexer(gen.split('|'))
    dummies2.iloc[i,indices] = 1

In [224]:
movies_windic = movies.join(dummies2.add_prefix('Genr_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genr_Animation                                1
Genr_Children's                               1
Genr_Comedy                                   1
Genr_Adventure                                0
Genr_Fantasy                                  0
Genr_Romance                                  0
Genr_Drama                                    0
Genr_Action                                   0
Genr_Crime                                    0
Genr_Thriller                                 0
Genr_Horror                                   0
Genr_Sci-Fi                                   0
Genr_Documentary                              0
Genr_War                                      0
Genr_Musical                                  0
Genr_Mystery                                  0
Genr_Film-Noir                                0
Genr_Western                            

In [225]:
np.random.seed(10)
values = np.random.rand(10)
bins = [0,0.2,0.4,0.6,0.8,1]
pd.get_dummies(pd.cut(values,bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,1,0
1,1,0,0,0,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,1,0,0
5,0,1,0,0,0
6,1,0,0,0,0
7,0,0,0,1,0
8,1,0,0,0,0
9,1,0,0,0,0


# string manipulation

In [260]:
val = 'a,b,  Guitar'
val.split(',')

['a', 'b', '  Guitar']

In [243]:
s10 = [x.lstrip() for x in val.split(',')]
s10

['a', 'b', 'guitar']

In [230]:
a, b, c = s10
a + "::" + b  + "::" +c

'a::b::guitar'

In [231]:
"::".join(s10)

'a::b::guitar'

In [232]:
'guitar' in val

True

In [233]:
val.index(',')

1

In [253]:
val.rfind('a')

10

In [235]:
val.index(':')

ValueError: substring not found

In [236]:
val.count(',')

2

In [240]:
val.replace({',':'::'},inplace = True)

TypeError: replace() takes no keyword arguments

In [261]:
val.startswith('a,')

True

In [263]:
val.casefold() # 处理其它语言：将大写转换成小写

'a,b,  guitar'

In [266]:
val.ljust(10,'#') # 填充，让字符串变成指定长度

'a,b,  Guitar'

In [317]:
text = 'a   b\t   dog'
re.split('\s+',text)
pattern = re.compile('\s+')
pattern.split(text)

['a', 'b', 'dog']

In [272]:
pattern.findall(text)

['   ', '\t   ']

In [279]:
pattern.search(text)

<_sre.SRE_Match object; span=(1, 4), match='   '>

In [285]:
print(re.match('a,',text))

None


In [286]:
pattern.sub('REDACTED',text) # 把pattern中匹配到的地方全部替换为对应的字符串

'aREDACTEDbREDACTEDdog'

In [315]:
pattern2 = re.compile('[a-z]+')
pattern2.findall('a')

['a']

In [330]:
data = {'Dave':'dave@google.com','Steve':'steve@gmail.com',
        'Rob':'rob@gmail.com','Wes':np.nan}
data = Series(data)

In [344]:
pattern3 = re.compile(r'([a-z]+)@(g[a-z]+)\.(com)')
matches = data.str.match(pattern3)

In [348]:
matches.str[0]

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64

In [350]:
data.str.repeat(2)

Dave     dave@google.comdave@google.com
Rob          rob@gmail.comrob@gmail.com
Steve    steve@gmail.comsteve@gmail.com
Wes                                 NaN
dtype: object